# Preprocesado de Datos
 ## Se importan datos usando las funcionalidades de Numpy y Pandas
 
 ### Trabajo de Fin de Grado - Universidad de Burgos
 #### Jesús García Armario

In [1]:
## Imports
import pandas as pd
import numpy as np

### Dataset Servicio Andaluz de Salud:

In [26]:
# Se importa el fichero excel de intervenciones realizadas
df = pd.read_excel('rptListadoIntervRealizadas.xls',header = [9,10], skipfooter = 12,index_col=[1,4,5])

# Eliminamos columnas vacías
df = df.droplevel(level=1,axis=0)
df = df.droplevel(level = 1, axis = 1)
df = df.dropna(axis=1, how='all')
del df['QUIRÓFANO']
del df['APELLIDOS, NOMBRE']
del df['TURNO']
del df['INICIO']
del df['FIN']
df = df.drop(df.columns[[0]],axis=1)

# Se renombran las columnas repetidas
cols = []
count = 1
countT = 1
contC = 1
for i in df.columns:
    if i=='DURACIÓN':
        cols.append('DURACIÓN'+str(count))
        count += 1
        continue
    if i == 'TIPO':
        cols.append('TIPO'+str(countT))
        countT+=1
        continue
    if i=='CARÁCTER ECONÓMICO':
        cols.append('CARÁCTER ECONÓMICO'+str(contC))
        contC+=1
        continue
    cols.append(i)

df.columns = cols

# Eliminamos la columna que sobra
del df['DURACIÓN1']
# Renombramos la otra
df.rename(columns = {"TIPO1": "TIPO", "TIPO2": "TURNO", "DURACIÓN2":"DURACIÓN", "CARÁCTER ECONÓMICO1":"CARÁCTER ECONÓMICO",
"CARÁCTER ECONÓMICO2":"PONDERACIÓN"},inplace=True)

# Eliminamos filas vacías
df = df.dropna(axis=0,how = 'all')
# Codificamos: Cirugía Menor = 0; Cirugía Mayor = 1
df['TIPO'] = df['TIPO'].replace(['Menor','Mayor'],[0,1])
# Codificamos: Mañana = 0; Tarde = 1
df['TURNO'] = df['TURNO'].replace(['Mañana','Tarde'],[0,1])
# Codificamos: Actividad Ordinaria = 0; Continuidad Asistencial = 1
df['CARÁCTER ECONÓMICO'] = df['CARÁCTER ECONÓMICO'].replace(['Actividad Ordinaria','Continuidad Asistencial'],[0,1])
# Eliminamos texto
df['TIPO'] = df['TIPO'].replace(r'^\D',np.nan,regex=True)
df['CARÁCTER ECONÓMICO'] = df['CARÁCTER ECONÓMICO'].replace(r'^\D',np.nan,regex=True)
df['PONDERACIÓN'] = df['PONDERACIÓN'].replace(r'^\D',np.nan,regex=True)
df['NUHSA'] = df['NUHSA'].replace(r'^\D',np.nan,regex=True)
# Eliminamos filas nulas
df = df.dropna(axis=0,how = 'all')

# Cambiamos el nombre del atributo NUHSA
df.rename(columns={"NUHSA":"INTERVENCIÓN"},inplace=True)
df.reset_index(inplace = True)
del df["level_0"]
df.rename(columns={"level_1":"NHC"},inplace = True)
df = df.groupby('NHC', as_index = False).aggregate({"INTERVENCIÓN":"last","TIPO":"first", "TURNO":"first", 
"CARÁCTER ECONÓMICO":"last","PONDERACIÓN":"last", "DURACIÓN":"first"})

# Eliminamos los valores que tienen algún nulo
df = df.dropna(axis=0,how='any')

EXPORTAMOS A ARCHIVO EXTERNO

In [27]:
# Exportamos a Excel
archivo = "ListadoInterv_Preprocesado.xlsx"

df.to_excel(archivo)

## Dataset Kaggle:

In [4]:
df_kaggle = pd.read_excel('./archive/2022_Q1_OR_Utilization.xlsx')
print(df_kaggle.head(5))

   Encounter ID       Date  OR Suite      Service  CPT Code  \
0         10001 2022-01-03         1     Podiatry     28110   
1         10002 2022-01-03         1     Podiatry     28055   
2         10003 2022-01-03         1     Podiatry     28297   
3         10004 2022-01-03         1     Podiatry     28296   
4         10005 2022-01-03         2  Orthopedics     27445   

                             CPT Description  Booked Time (min)  \
0   Partial ostectomy, fifth metatarsal head                 90   
1  Neurectomy, intrinsic musculature of foot                 60   
2                       Lapidus bunionectomy                150   
3         Bunionectomy with distal osteotomy                120   
4        Arthroplasty, knee, hinge prothesis                120   

          OR Schedule           Wheels In          Start Time  \
0 2022-01-03 07:00:00 2022-01-03 07:05:00 2022-01-03 07:32:00   
1 2022-01-03 08:45:00 2022-01-03 09:48:00 2022-01-03 10:13:00   
2 2022-01-03 10:00:00 2